In [1]:
!pip -q install -U ultralytics iterative-stratification

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00:00:0100:01


In [2]:
%%writefile convert_npy_to_png.py
#src/features/convert_npy_to_png.py
import numpy as np
import os
from tqdm import tqdm
from pathlib import Path
from PIL import Image

def normalize_to_uint8(img):
    img = np.clip(img, 0, 10000)
    img = img / 10000.0
    img = (img * 255).astype(np.uint8)
    return img

def convert_npy_to_png(npy_folder, output_folder, bands=[3,2,1]):
    npy_folder = Path(npy_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)
    for filename in tqdm(os.listdir(npy_folder), desc=f"Converting {npy_folder.name}"):
        if filename.endswith(".npy"):
            path = npy_folder / filename
            data = np.load(path)
            rgb = data[:, :, bands]
            rgb = normalize_to_uint8(rgb)
            image_pil = Image.fromarray(rgb)
            out_path = output_folder / filename.replace(".npy", ".png")
            image_pil.save(out_path)

Writing convert_npy_to_png.py


In [3]:
%%writefile config_yolo.py
#src/utils/config_yolo.py
import os
import random
import numpy as np
import torch

class CFG:
    seed = 42
    batch=20
    random_state = 42
    folds = 10
    epochs= 100
    imgsz=320
    device=0
    optimizer='AdamW'
    lr0=3e-4
    momentum=0.9
    weight_decay=1e-2
    close_mosaic=30
    seed=42
    patience=10
    threshold=0.5

def seed_everything(seed=CFG.seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)

Writing config_yolo.py


In [4]:
%%writefile train_yolo.py
# src/models/train_yolo.py
import os
import shutil
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from config_yolo import CFG, seed_everything

seed_everything(CFG.seed)

# === Helper functions ===
def prepare_classification_dir(df, base_dir):
    """Copy images into YOLO classification folder structure."""
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f'Preparing {base_dir.name}'):
        img_path = Path(row['image_path'])
        class_name = str(row['new_class'])
        class_dir = base_dir / class_name
        class_dir.mkdir(parents=True, exist_ok=True)
        shutil.copy2(img_path, class_dir / img_path.name)

def prepare_test_dir(df, base_dir):
    """Copy test images into a flat directory."""
    for img_path in tqdm(df['image_path'].unique(), desc=f'Preparing test'):
        shutil.copy2(img_path, base_dir / Path(img_path).name)

# === Main training function ===
def train_yolo(
    train_csv,
    test_csv,
    images_dir,
    model_dir,
    folds=CFG.folds,
    epochs=CFG.epochs,
    imgsz=CFG.imgsz,
    device=CFG.device,
    batch=CFG.batch,
    optimizer=CFG.optimizer,
    lr0=CFG.lr0,
    momentum=CFG.momentum,
    weight_decay=CFG.weight_decay,
    close_mosaic=CFG.close_mosaic,
    seed=CFG.seed,
    patience=CFG.patience
):
    # Load train/test
    train = pd.read_csv(train_csv)
    test = pd.read_csv(test_csv)

    # Map class labels to indices
    unique_classes = sorted(train['label'].unique())
    full_label_dict = {cls: idx for idx, cls in enumerate(unique_classes)}
    train['new_class'] = train['label'].map(full_label_dict)

    # Group multilabels
    grouped = train.groupby('ID')['new_class'].apply(list).reset_index()
    for cls in unique_classes:
        grouped[cls] = -1
    reverse_mapping = {v: k for k, v in full_label_dict.items()}
    for idx, labels in enumerate(grouped['new_class']):
        for lbl in set(labels):
            grouped.loc[idx, reverse_mapping[lbl]] = 1

    # Multilabel stratified folds
    grouped['fold'] = -1
    mskf = MultilabelStratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    for i, (_, val_idx) in enumerate(mskf.split(grouped[['ID']], grouped[unique_classes])):
        grouped.loc[val_idx, 'fold'] = i

    # Add image paths
    grouped['image_path'] = grouped['ID'].apply(lambda x: Path(images_dir) / f"{x}.png")
    test['image_path'] = test['ID'].apply(lambda x: Path(images_dir.replace('train', 'test')) / f"{x}.png")

    # Loop over folds
    for fold in range(folds):
        print(f"\n🚀 Training Fold {fold + 1}/{folds}")

        # Define directories
        fold_root = Path(model_dir) / f'fold_{fold + 1}'
        train_dir = fold_root / 'train'
        val_dir = fold_root / 'val'
        test_dir = Path(model_dir) / 'test/images'

        # Clean dirs
        for d in [train_dir, val_dir, test_dir]:
            if d.exists():
                shutil.rmtree(d)
            d.mkdir(parents=True, exist_ok=True)

        # Prepare data for YOLO classification
        prepare_classification_dir(grouped[grouped['fold'] != fold], train_dir)
        prepare_classification_dir(grouped[grouped['fold'] == fold], val_dir)
        prepare_test_dir(test, test_dir)

        # Train YOLO
        model = YOLO("yolo11l-cls.pt")
        model.train(
            data=str(fold_root),
            epochs=epochs,
            imgsz=imgsz,
            device=device,
            batch=batch,
            optimizer=optimizer,
            lr0=lr0,
            momentum=momentum,
            weight_decay=weight_decay,
            close_mosaic=close_mosaic,
            seed=seed,
            patience=patience,
            project=str(model_dir),
            name=f"fold_{fold + 1}",
            exist_ok=True
        )

        # Save best weights to fold-specific directory
        src = Path(model_dir) / f"fold_{fold + 1}/weights/best.pt"
        dst = fold_root / "best.pt"
        shutil.copy(src, dst)
        print(f"✅ Fold {fold + 1} model saved at {dst}")

Writing train_yolo.py


In [11]:
%%writefile infer_yolo.py
#src/models/infer_yolo.py
import os
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
from config_yolo import seed_everything, CFG
from tqdm import tqdm

def infer_yolo(test_csv,test_dir, model_dir, output_path, folds=CFG.folds, threshold=0.5):
    test = pd.read_csv(test_csv)
    test_images_dir = Path(test_dir)
    image_files = sorted(os.listdir(test_images_dir))
    
    all_probs = []
    for fold in range(1, folds+1):
        model = YOLO(str(Path(model_dir) / f'fold_{fold}/best.pt'))
        fold_probs = []
        for img in tqdm(image_files, desc=f"Fold {fold} inference"):
            results = model(test_images_dir / img, verbose=False, imgsz=CFG.imgsz)
            probs = results[0].probs.data.cpu().numpy()
            fold_probs.append(float(probs[1]))  # Probability of class 1
        all_probs.append(fold_probs)

    # Average probabilities
    probs_matrix = np.array(all_probs).T
    avg_probs = probs_matrix.mean(axis=1)
    preds = (avg_probs > threshold).astype(int)
    
    # Save
    df = pd.DataFrame({
        'ID': [f.replace('.png', '') for f in image_files],
        'Probs': avg_probs,
        'Predicted': preds
    })
    df.to_csv(output_path, index=False)
    print(f"✅ Saved predictions to {output_path}")


Overwriting infer_yolo.py


In [ ]:
%%writefile main_yolo.py
import argparse
from pathlib import Path
from convert_npy_to_png import convert_npy_to_png
from train_yolo import train_yolo
from infer_yolo import infer_yolo

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", choices=["train", "infer"], required=True)
    parser.add_argument("--train_csv", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/Train.csv")
    parser.add_argument("--test_csv", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/Test.csv")
    parser.add_argument("--train_npy_dir", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/train_data/train_data")
    parser.add_argument("--test_npy_dir", type=str, default="/kaggle/input/slideandseekclasificationlandslidedetectiondataset/test_data/test_data")
    parser.add_argument("--image_dir", type=str, default="train_data_sentinel_png")
    parser.add_argument("--test_dir", type=str, default="test_data_sentinel_png")
    parser.add_argument("--model_dir", type=str, default="models/yolo")
    parser.add_argument("--output_path", type=str, default="yolo_submission_probs_labels.csv")
    args = parser.parse_args()

    if args.mode == "train":
        # Step 1: Convert npy → png (if not already done)
        if not Path(args.image_dir).exists() or len(list(Path(args.image_dir).glob("*.png"))) == 0:
            print(f"⚙️ Converting training npy files from {args.train_npy_dir} to PNG at {args.image_dir}")
            convert_npy_to_png(args.train_npy_dir, args.image_dir)
        if not Path(args.test_dir).exists() or len(list(Path(args.test_dir).glob("*.png"))) == 0:
            print(f"⚙️ Converting test npy files from {args.test_npy_dir} to PNG at {args.test_dir}")
            convert_npy_to_png(args.test_npy_dir, args.test_dir)

        # Step 2: Train YOLO models
        train_yolo(args.train_csv, args.test_csv, args.image_dir, args.model_dir)

    elif args.mode == "infer":
        infer_yolo(args.test_csv,args.test_dir, args.model_dir, args.output_path)

Writing main_yolo.py


In [7]:
!python main_yolo.py --mode train 

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
⚙️ Converting training npy files from /kaggle/input/slideandseekclasificationlandslidedetectiondataset/train_data/train_data to PNG at train_data_sentinel_png
Converting train_data: 100%|████████████████| 7147/7147 [01:33<00:00, 76.04it/s]
⚙️ Converting test npy files from /kaggle/input/slideandseekclasificationlandslidedetectiondataset/test_data/test_data to PNG at test_data_sentinel_png
Converting test_data: 100%|█████████████████| 5397/5397 [01:05<00:00, 82.66it/s]

🚀 Training Fold 1/2
Preparing test: 100%|█████████████████████| 5397/5397 [00:00<00:00, 8741.74it/s]
Ultralytics 8.3.174 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=Fa

In [12]:
!python main_yolo.py --mode infer

Fold 2 inference: 100%|█████████████████████| 5397/5397 [01:04<00:00, 84.02it/s]
✅ Saved predictions to yolo_submission.csv
